# Confluências, modelo de mommentum

Confluências-Sum é um modelo de mommentum idealizado para captar oportunidades de curto prazo (3 a 7 dias), se baseando na confluencîa de alguns fatores para determinar compra ou venda de um ativo.

Esse algoritmo foi desenhado para performar no mercado cripto, devemos nos atentar par possiveis desvios de comportamento.

In [1]:
# Bibliotecas necessárias
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_datareader as web
from datetime import datetime
import xlwt
import matplotlib.pyplot as plt



Ativos:

In [2]:
tickers = "VALE3.SA"

In [3]:
end_date =  '{}-{}-{}' .format( datetime.now().year,  datetime.now().month,  datetime.now().day)

In [4]:
yahoo_data = yf.download(tickers=tickers, #
start="2022-08-29", 
end=end_date, interval='1d')
intraday_data = yf.download(tickers=tickers, period="1d", interval="1h")


 In case we have to verifeiy it better
 print(yahoo_data)

In [ ]:
close = yahoo_data['Close']
volume = yahoo_data['Volume']
high = yahoo_data['High']
low = yahoo_data['Low']

# Intraday data
intraday_close = intraday_data['Close']
intraday_high = intraday_data['High']
intraday_low = intraday_data['Low']
intraday_volume = intraday_data['Volume']


In [ ]:
dados = pd.DataFrame()
dados['Volume'] = volume.iloc[:-1]
dados['Close'] = close.iloc[:-1]
dados['Intraday Close'] = intraday_close.iloc[:-1]
MME3 = close.ewm(span=9).mean()
MME7 = close.ewm(span=21).mean()
dados['MME3'] = MME3.iloc[:-1]
dados['MME7'] = MME7.iloc[:-1]
both_close =  pd.concat([close,intraday_close.tail(1)]) 


In [ ]:
def Volume_Projetado(VolumeAtual):
  now = datetime.now()
  BrazilHour = now.hour -3
  BrazilMinute = now.minute
  diffMinutes = 60 - BrazilMinute

  diffClose = (18 - BrazilHour) * 60 - BrazilMinute
  diffOpen = (BrazilHour - 10) * 60 + BrazilMinute

  VolumeFaltante = diffClose*(VolumeAtual/diffOpen)

  VolumeProjetado = VolumeAtual + VolumeFaltante
  return VolumeProjetado

In [ ]:
# DELTA AVAT
# Volume Projetado
last_two = yf.download(tickers=tickers, period='3d', interval='1d')['Volume'].iloc[:-1]
last_two.iloc[:-1] = Volume_Projetado(last_two.iloc[:-1] )
last_two

[*********************100%***********************]  1 of 1 completed


Date
2022-08-26    2.443723e+07
2022-08-29    2.212810e+07
Name: Volume, dtype: float64

In [ ]:
### RETORNO CLOSE ANTERIOR E ULTIMO PREÇO
dados['Retorno (Close[1]/Intra)'] = ((dados['Intraday Close'] / dados['Close'] ) -1 ) * 100

In [ ]:
#### DELTA AVAT 1D
# VOLUME PROJETADO / 1D AVAT
delta_avat_1d = (last_two.iloc[:-1] / last_two.iloc[:-2]) - 1
dados['Volume Projetado'] = last_two.iloc[:-1]
dados['Delta AVAT 1D'] = (dados['Volume Projetado']/dados['Volume']) - 1
dados['Fast AVAT'] = (dados['Delta AVAT 1D'] > 0.5) & (dados['Retorno (Close[1]/Intra)'] < 0.5 )  & (dados['Retorno (Close[1]/Intra)'] > -0.5 )
dados['Fast AVAT'] = dados['Fast AVAT'].astype(int)

In [ ]:
print(dados['Volume Projetado'])

2022-08-30 10:00:00-03:00   NaN
2022-08-30 11:00:00-03:00   NaN
2022-08-30 12:00:00-03:00   NaN
2022-08-30 13:00:00-03:00   NaN
2022-08-30 14:00:00-03:00   NaN
2022-08-30 15:00:00-03:00   NaN
2022-08-30 16:00:00-03:00   NaN
Name: Volume Projetado, dtype: float64


In [ ]:
MME3_intraday = both_close.ewm(span=3).mean()
MME7_intraday = both_close.ewm(span=7).mean()


In [ ]:
dados['MME3_intraday'] = MME3_intraday.iloc[-2]
dados['MME7_intraday'] = MME7_intraday.iloc[-2]

In [ ]:
dados['Fast EMA Cross'] = (dados['MME3'] <  dados['MME3_intraday'])  & ( dados['MME3_intraday'] > dados['MME7_intraday']) & (dados['MME3'] < dados['MME7'])
dados['Fast EMA Cross Minus'] =(dados['MME3'] >  dados['MME3_intraday'])  & ( dados['MME3_intraday'] < dados['MME7_intraday']) & (dados['MME3'] > dados['MME7'])

display(dados)

,Volume,Close,Intraday Close,MME3,MME7,Retorno (Close[1]/Intra),Volume Projetado,Delta AVAT 1D,Fast AVAT,MME3_intraday,MME7_intraday,Fast EMA Cross,Fast EMA Cross Minus
2022-08-30 10:00:00-03:00,NaN,NaN,65.449997,NaN,NaN,NaN,NaN,NaN,0,66.910004,66.910004,False,False
2022-08-30 11:00:00-03:00,NaN,NaN,65.019997,NaN,NaN,NaN,NaN,NaN,0,66.910004,66.910004,False,False
2022-08-30 12:00:00-03:00,NaN,NaN,64.930000,NaN,NaN,NaN,NaN,NaN,0,66.910004,66.910004,False,False
2022-08-30 13:00:00-03:00,NaN,NaN,64.720001,NaN,NaN,NaN,NaN,NaN,0,66.910004,66.910004,False,False
2022-08-30 14:00:00-03:00,NaN,NaN,64.500000,NaN,NaN,NaN,NaN,NaN,0,66.910004,66.910004,False,False
2022-08-30 15:00:00-03:00,NaN,NaN,64.739998,NaN,NaN,NaN,NaN,NaN,0,66.910004,66.910004,False,False
2022-08-30 16:00:00-03:00,NaN,NaN,64.720001,NaN,NaN,NaN,NaN,NaN,0,66.910004,66.910004,False,False
